In [4]:
import pandas as pd
from random import randint, choice
from scipy.stats import binom
from numpy.random import choice
from tqdm.notebook import tqdm

In [5]:
dt = pd.read_csv('forecast1.csv', sep=';')
dt

,day_temp,day_pressure,day_obl,day_phen,day_dir,day_wind,areal,city,year,month,...,красноперка,налим,густера,амур,ерш,сазан,подуст,толстолобик,вобла,хариус
0,-4,749,dull,snow,Ю,3,Алтайский край,Барнаул,2020,1,...,-1,-1.0,-1,-1,-1,-1.0,-1,-1,-1,-1.0
1,-1,750,dull,-,Ю,5,Алтайский край,Барнаул,2020,1,...,-1,-1.0,-1,-1,-1,-1.0,-1,-1,-1,-1.0
2,-3,749,dull,-,Ю,2,Алтайский край,Барнаул,2020,1,...,-1,-1.0,-1,-1,-1,-1.0,-1,-1,-1,-1.0
3,-6,753,dull,-,Ю,1,Алтайский край,Барнаул,2020,1,...,-1,-1.0,-1,-1,-1,-1.0,-1,-1,-1,-1.0
4,-2,752,suncl,-,Ю,3,Алтайский край,Барнаул,2020,1,...,-1,-1.0,-1,-1,-1,-1.0,-1,-1,-1,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30516,-6,752,dull,-,З,1,Московская область,Щелково,2020,12,...,-1,-1.0,-1,-1,-1,-1.0,-1,-1,-1,-1.0
30517,-6,759,sunc,-,ЮВ,1,Московская область,Щелково,2020,12,...,-1,-1.0,-1,-1,-1,-1.0,-1,-1,-1,-1.0
30518,-5,754,dull,-,ЮВ,3,Московская область,Щелково,2020,12,...,-1,-1.0,-1,-1,-1,-1.0,-1,-1,-1,-1.0
30519,-3,753,dull,-,-,0,Московская область,Щелково,2020,12,...,-1,-1.0,-1,-1,-1,-1.0,-1,-1,-1,-1.0


In [77]:
class DayGenerator:
    def __init__(self, dataframe):
        self.dataframe = dataframe
        self.times = [12, 15, 18, 21, 0, 3, 6, 9]
        self.dirs = {
            'С': {'values': ['СЗ', 'СВ'], 'p': [0.2, 0.3]},
            'СЗ': {'values': ['С', 'З'], 'p': [0.25, 0.25]},
            'З': {'values': ['СЗ', 'ЮЗ'], 'p': [0.2, 0.3]},
            'ЮЗ': {'values': ['З', 'Ю'], 'p': [0.25, 0.25]},
            'Ю': {'values': ['ЮЗ', 'ЮВ'], 'p': [0.3, 0.2]},
            'ЮВ': {'values': ['Ю', 'В'], 'p': [0.2, 0.3]},
            'В': {'values': ['СВ', 'ЮВ'], 'p': [0.3, 0.2]},
            'СВ': {'values': ['С', 'В'], 'p': [0.3, 0.2]},
        }
        
        self.phens = {
            'snow': {'values': ['небольшой снег', 'снег', 'снег с дождём', 'сильный снег', 'мокрый снег'], 'p': [0.35, 0.25, 0.15, 0.1, 0.15]},
            'rain': {'values': ['небольшой дождь', 'дождь', 'сильный дождь'], 'p': [0.45, 0.35, 0.2]},
            'storm': {'values': ['небольшой дождь', 'дождь', 'гроза', 'сильный дождь'], 'p': [0.15, 0.25, 0.3, 0.3]},
        }
        
        self.obl = {
            'dull': {'values': ['пасмурно', 'облачно'], 'p': [0.7, 0.3]},
            'suncl': {'values': ['облачно', 'малооблачно', 'пасмурно'], 'p': [0.6, 0.3, 0.1]},
            'sun': {'values': ['ясно', 'малооблачно'], 'p': [0.7, 0.3]},
            'sunc': {'values': ['малооблачно', 'ясно', 'облачно'], 'p': [0.6, 0.2, 0.2]},
        }
        
        self.fishs = ['щука', 'судак', 'окунь', 'берш', 'речная форель', 'озерная форель', 'елец', 'чехонь', 'сом', 'голавль', 'язь',
         'карп', 'жерех', 'лещ', 'карась', 'линь', 'пескарь', 'ротан', 'плотва', 'красноперка', 'налим', 'густера',
         'амур', 'ерш', 'сазан', 'подуст', 'толстолобик', 'вобла', 'хариус']
        
    def __getitem__(self, idx):
        day_priew = self.dataframe.loc[[idx - 1]]
        day = self.dataframe.loc[[idx]]
        day_next = self.dataframe.loc[[idx + 1]]
        pressure = ','.join(map(str, self.gen_pressure(day_priew['day_pressure'].item(), day['day_pressure'].item(), day_next['day_pressure'].item())))
        temperature = ','.join(map(str, self.gen_temperature(day_priew['day_temp'].item(), day['day_temp'].item(), day_next['day_temp'].item())))
        wind_, gust_ = self.gen_wind(day_priew['day_wind'].item(), day['day_wind'].item(), day_next['day_wind'].item())
        wind = ','.join(map(str, wind_))
        gust = ','.join(map(str, gust_))
        wind_direction = ','.join(self.gen_dir(day['day_dir'].item()))
        phenomen_= self.gen_phenomen(day['day_obl'].item(), day['day_phen'].item())
        humidity = ','.join(map(str,self.gen_hum(phenomen_)))
        phenomen = ','.join(phenomen_)
        uv_index = ','.join(map(str,self.gen_uv(day['month'].item())))
        moon_direction, moon = self.gen_moon(sum(day[fish].item() for fish in self.fishs) / len(self.fishs))
        return {
            'pressure': pressure,
            'temperature': temperature,
            'wind': wind,
            'gust': gust,
            'wind_direction': wind_direction,
            'humidity': humidity,
            'phenomenon': phenomen,
            'uv_index': uv_index,
            'moon_direction': moon_direction,
            'moon': moon,
            'month': day['month'].item(),
            'day': day['day'].item(),
            
        }
        
    def dist_(self, left, right, len_):
        is_revert = False
        if left > right:
            left, right = right, left
            is_revert = True
        result = []
        current = left
        for _ in range(len_ - 1): 
            sub = binom.rvs(right - current, 1 / len_)
            current += sub
            result.append(current)
        result.append(right)
        return result[::-1] if is_revert else result
        
    def gen_pressure(self, pressure_priew, pressure, pressure_next):
        pressure_ = {}
        priew_sub = pressure - pressure_priew
        next_sub = pressure_next - pressure
        current_pressure = pressure_priew
        for time in [15, 18, 21]:
            sub = randint(-2, 0) if priew_sub < 0 else randint(-1, 1)
            current_pressure += sub
        dist = self.dist_(current_pressure, pressure, 5)
        for idx, time in enumerate([0, 3, 6, 9, 12]):
            pressure_.update({time: dist[idx]})
        current_pressure = pressure
        for time in [15, 18, 21]:
            sub = randint(-2, 0) if next_sub < 0 else randint(-1, 1)
            current_pressure += sub
            pressure_.update({time: current_pressure})
        return [pressure_[key] for key in sorted(pressure_)]
    
    def gen_wind(self, wind_priew, wind, wind_next):
        wind_ = {}
        priew_sub = wind - wind_priew
        next_sub = wind_next - wind
        current_wind = randint(0, randint(wind_priew, wind)) if priew_sub > 0 else 0
        dist = self.dist_(current_wind, wind, 5)
        for idx, time in enumerate([0, 3, 6, 9, 12]):
            wind_.update({time: dist[idx]})
        current_wind = wind_[12] + randint(-1, 2)
        current_wind = max(0, current_wind)
        wind_.update({15: current_wind})
        current_wind += randint(-1, 2)
        current_wind = max(0, current_wind)
        wind_.update({18: current_wind})
        current_wind += randint(-3, 0)
        current_wind = max(0, current_wind)
        wind_.update({21: current_wind})
        gust_= {time: wind_[time] + randint(3, 8) for time in self.times}
        return [wind_[key] for key in sorted(wind_)], [gust_[key] for key in sorted(gust_)]
    
    def gen_dir(self,direction):
        dir_ = {}
        if direction == '-':
            direction = choice(list(self.dirs.keys()))
        dist = list(choice([direction] + self.dirs[direction]['values'], 8, p=[0.5] + self.dirs[direction]['p']))
        for idx, time in enumerate(self.times):
            dir_.update({time: dist[idx]})
        return [dir_[key] for key in sorted(dir_)]
    
    def gen_temperature(self, temp_priew, temp, temp_next):
        temp_ = {}
        priew_sub = temp - temp_priew
        next_sub = temp_next - temp
        current_temp = temp_priew
        for time in [15, 18, 21]:
            sub = randint(-3, -1) if priew_sub < 0 else randint(-2, 0)
            current_temp += sub
        dist = self.dist_(current_temp, temp, 5)
        for idx, time in enumerate([0, 3, 6, 9, 12]):
            temp_.update({time: dist[idx]})
        current_temp = temp
        for time in [15, 18, 21]:
            sub = randint(-3, -1) if next_sub < 0 else randint(-2, 0)
            current_temp += sub
            temp_.update({time: current_temp})
        return [temp_[key] for key in sorted(temp_)]
    
    def gen_phenomen(self, obl, phen):
        phens_ = {}
        if obl == '-':
            obl = 'sun'
        dist_olb = list(choice(self.obl[obl]['values'], 8, p=self.obl[obl]['p']))
        if phen == '-':
            for idx, time in enumerate(self.times):
                phens_.update({time: dist_olb[idx]})
            return [phens_[key] for key in sorted(phens_)] 
        dist_phens = list(choice(self.phens[phen]['values'], 8, p=self.phens[phen]['p']))
        for idx, time in enumerate(self.times):
            phens_.update({time: '.'.join((dist_olb[idx], dist_phens[idx]))})
        return [phens_[key] for key in sorted(phens_)]
    
    def gen_hum(self, phens):
        hum_ = []
        current_hum = randint(30, 70)
        for phen in phens:
            if 'дождь' in phen:
                current_hum += randint(0, 20)
                current_hum = min(current_hum, randint(93, 99))
            else:
                current_hum += randint(-10, 5)
                current_hum = max(current_hum, randint(10, 29))
            hum_.append(current_hum)
        return hum_
    
    def gen_uv(self, month):
        uv_ = {}
        if month in [12, 1, 2]:
            uv_ = {0: 0, 3: 0, 6: 0, 9: 0, 12: randint(1, 2), 15: 1, 18: 0, 21:0} 
        elif month in [3, 11, 10, 4]:
            uv_ = {0: 0, 3: 0, 6: 0, 9: 1, 12: randint(2, 3), 15: randint(1, 2), 18: 1, 21: 0}
        elif month in [5, 9]:
            uv_ = {0: 0, 3: 0, 6: randint(0, 1), 9: randint(1, 2), 12: randint(2, 4), 15: randint(2, 3), 18: randint(1, 2), 21: randint(0, 1)}
        else:
            uv_ = {0: 0, 3: 0, 6: randint(1, 2), 9: randint(1, 3), 12: randint(2, 4), 15: randint(1, 3), 18: randint(1, 3), 21: randint(1, 2)}
        return [uv_[key] for key in sorted(uv_)]
    
    def gen_moon(self, forecast):
        if forecast > 0.5:
            return 2* randint(0, 1) - 1, randint(35, 70)
        else:
            if randint(0, 1):
                return 2* randint(0, 1) - 1, randint(0, 35)
            else:
                return 2* randint(0, 1) - 1, randint(70, 99)   

In [83]:
ALONE_KEYS = {'time', 'day', 'month', 'humidity', 'uv_index', 'moon', 'moon_direction'}
DIGIT_KEYS = {'temperature', 'wind', 'gust', 'pressure', 'humidity', 'uv_index'}
CATEGORY_KEYS = {'phenomenon', 'wind_direction'}
MOON_KEYS = {'moon', 'moon_direction'}
SUN_KEYS = {'sun_up', 'sun_down'}
WIND_DIRECTIONS = ['Ю', 'ЮЗ', 'З', 'СЗ', 'С', 'СВ', 'В', 'ЮВ']
PHENOMENONS = ['ясно', 'малооблачно', 'облачно', 'пасмурно', 'небольшой дождь', 'дождь', 'сильный дождь',
               'небольшой снег', 'снег', 'снег с дождём', 'сильный снег', 'гроза', 'мокрый снег']
REGIONS = ['Алтайский край', 'Амурская область', 'Архангельская область', 'Астраханская область',
           'Белгородская область', 'Брянская область', 'Владимирская область', 'Волгоградская область',
           'Вологодская область', 'Воронежская область', 'Еврейская автономная область', 'Забайкальский край',
           'Ивановская область', 'Иркутская область', 'Кабардино-Балкарская республика', 'Калининградская область',
           'Калужская область', 'Камчатский край', 'Карачаево-Черкесская республика', 'Кемеровская область',
           'Кировская область', 'Костромская область', 'Краснодарский край', 'Красноярский край', 'Курганская область',
           'Курская область', 'Ленинградская область', 'Липецкая область', 'Магаданская область', 'Московская область']


num_hours = 8
num_days = 3

def preprocess_(data):
    all_data = {}
    for d in data:
        for key in DIGIT_KEYS:
            temp = list(map(int, d[key].split(',')))
            if key in all_data:
                all_data[key] = all_data[key] + temp
            else:
                all_data[key] = temp
        for key in MOON_KEYS:
            temp = [d[key] for _ in range(num_hours)]
            if key in all_data:
                all_data[key] = all_data[key] + temp
            else:
                all_data[key] = temp
        for key in CATEGORY_KEYS:
            if key in all_data:
                all_data[key] = all_data[key] + d[key].split(',')
            else:
                all_data[key] = d[key].split(',')
        days = [d['day'] for _ in range(num_hours)]
        months = [d['month'] for _ in range(num_hours)]
        if 'day' in all_data:
            all_data['day'] = all_data['day'] + days
        else:
            all_data['day'] = days
        if 'month' in all_data:
            all_data['month'] = all_data['month'] + months
        else:
            all_data['month'] = months
        if 'time' in all_data:
            all_data['time'] = all_data['time'] + list(range(0, num_hours * 3, 3))
        else:
            all_data['time'] = list(range(0, num_hours * 3, 3))
    return all_data

def slice_(data, left_bound, righ_bound):
    slice_data = {
        key: data[key][left_bound: righ_bound] for key in data
    }
    return slice_data

def preprocess_batch_(data):
    vec = {}
    for key in data:
        if key in DIGIT_KEYS and not key in ALONE_KEYS:
            for i in range(len(data[key])):
                key_name = '{}_{}'.format(key, i)
                vec.update({key_name: data[key][i]})
        elif key == 'phenomenon':
            phenomenons_ = [_.split('.') for _ in data[key]]
            for phenomenon in PHENOMENONS:
                for i in range(len(phenomenons_)):
                    key_name = '{}_{}'.format(phenomenon, i)
                    vec.update({key_name: int(phenomenon in phenomenons_[i])})
        elif key == 'wind_direction':
            for wind_direction in WIND_DIRECTIONS:
                for i in range(len(data[key])):
                    key_name = '{}_{}'.format(wind_direction, i)
                    vec.update({key_name: int(wind_direction == data[key][i])})
        elif key == 'month':
            for month in range(1, 31):
                key_name = 'month_{}'.format(month)
                vec.update({key_name: int(month == data[key][-1])})
        elif key == 'time':
            for time in range(0, 24, 3):
                key_name = 'time_{}'.format(time)
                vec.update({key_name: int(time == data[key][-1])})
        elif key == 'moon_direction':
            for moon_direction in [-1, 1]:
                key_name = 'moon_direction_{}'.format(moon_direction)
                vec.update({key_name: int(moon_direction == data[key][-1])})
        elif key in ALONE_KEYS:
            key_name = '{}'.format(key)
            vec.update({key_name: data[key][-1]})
    return vec

def gen_forecast(forecast, time, fish):
    if fish in ['сом', 'налим']:
        if time in [21, 0, 3]:
            return 1 * forecast
        else:
            if forecast:
                if randint(1, 10) <= 5:
                    return 1
                return 0
            return 0
    else:
        if time in [6, 18]:
            if not forecast:
                if randint(1, 10) <= 5:
                    return 0
                return 1
            else:
                return 1
        elif time in [9, 12, 15]:
            if forecast:
                if randint(1, 10) <= 8:
                    return 1
                return 0
            else:
                if randint(1, 10) <= 2:
                    return 1
                return 0
        else:
            if forecast:
                if randint(1, 10) <= 5:
                    return 1
                return 0
            else:
                return 0
            

In [79]:
gen = DayGenerator(dt)

In [86]:
train_data = []
for idx, row in tqdm(dt.iterrows()):
    if row['щука'] > -1:
        data = dt[idx - 3: idx + 1]
        day = gen[idx]
        day_1 = gen[idx - 1]
        day_2 = gen[idx - 2]
        day_3 = gen[idx - 3]
        all_data = preprocess_([day_3, day_2, day_1, day])
        len_data = len(all_data['moon'])
        probs = {fish: [] for fish in gen.fishs}
        for fish in gen.fishs:
            for i in range(num_hours * num_days, len_data + 1):
                slice_data = slice_(all_data, i - num_hours * num_days, i)
                vec = preprocess_batch_(slice_data)
                for fish_ in gen.fishs:
                    vec.update({fish_: int(fish_ == fish)})
                time = 0
                for j in range(0, 24, 3):
                    if vec['time_{}'.format(j)] == 1:
                        time = j
                        break
                forecast = gen_forecast(row[fish], time, fish)
                vec.update({'forecast': forecast})
                train_data.append(vec)

C:\Users\Xiaomi\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\Xiaomi\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\Xiaomi\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\Xiaomi\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\Xiaomi\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\Xiaomi\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\Xiaomi\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: FutureWarning: `it

In [87]:
len(train_data)

13833

In [89]:
train_dt = pd.DataFrame(train_data)

In [91]:
train_dt.describe()

,temperature_0,temperature_1,temperature_2,temperature_3,temperature_4,temperature_5,temperature_6,temperature_7,temperature_8,temperature_9,...,налим,густера,амур,ерш,сазан,подуст,толстолобик,вобла,хариус,forecast
count,13833.000000,13833.000000,13833.000000,13833.000000,13833.000000,13833.000000,13833.000000,13833.000000,13833.000000,13833.000000,...,13833.000000,13833.000000,13833.000000,13833.000000,13833.000000,13833.000000,13833.000000,13833.000000,13833.000000,13833.000000
mean,7.578616,7.578616,7.584906,7.584906,7.716981,7.471698,7.262055,7.029350,7.100629,7.106918,...,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.385238
std,13.708617,13.587257,13.484371,13.366623,13.288674,13.187796,13.131187,13.098522,12.989835,12.897864,...,0.182472,0.182472,0.182472,0.182472,0.182472,0.182472,0.182472,0.182472,0.182472,0.486669
min,-25.000000,-25.000000,-24.000000,-22.000000,-22.000000,-22.000000,-22.000000,-22.000000,-22.000000,-22.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-2.000000,-2.000000,-2.000000,-2.000000,-1.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,18.000000,18.000000,18.000000,18.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,35.000000,35.000000,35.000000,35.000000,35.000000,33.000000,31.000000,31.000000,31.000000,31.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [92]:
train_dt = train_dt.sample(frac=1).reset_index(drop=True)

In [93]:
y = train_dt['forecast']
del train_dt['forecast']

In [94]:
train_dt

,temperature_0,temperature_1,temperature_2,temperature_3,temperature_4,temperature_5,temperature_6,temperature_7,temperature_8,temperature_9,...,красноперка,налим,густера,амур,ерш,сазан,подуст,толстолобик,вобла,хариус
0,21,21,22,22,24,23,21,18,17,16,...,0,0,0,0,0,0,0,0,0,0
1,3,1,0,3,4,5,5,7,6,4,...,0,0,0,0,0,0,0,0,0,0
2,15,14,13,15,15,16,16,18,18,16,...,0,0,0,0,0,0,0,0,0,0
3,-4,-7,-10,-13,-13,-12,-12,-12,-8,-10,...,0,0,0,0,0,0,0,0,0,0
4,-2,0,-1,-1,-3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13828,1,-2,-4,-7,-3,-3,-3,-3,-3,-5,...,0,0,0,0,0,0,0,0,0,0
13829,19,19,19,21,19,18,16,19,20,21,...,0,1,0,0,0,0,0,0,0,0
13830,19,19,20,21,19,17,15,16,16,18,...,0,0,0,0,0,0,0,0,0,0
13831,24,22,22,20,23,24,24,24,28,26,...,0,0,0,0,0,0,0,0,0,0


In [95]:
X = train_dt.values

In [96]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [97]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

In [98]:
model.fit(X_train, y_train)

C:\Users\Xiaomi\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [99]:
preds = model.predict(X_test)

In [100]:
from sklearn.metrics import accuracy_score

print(accuracy_score(preds, y_test))

0.7253342970726419


In [101]:
model.predict_proba(X_test)

array([[0.0194085 , 0.9805915 ],
       [0.77012626, 0.22987374],
       [0.68480069, 0.31519931],
       ...,
       [0.81462916, 0.18537084],
       [0.04466756, 0.95533244],
       [0.24628216, 0.75371784]])